# Import Libraries

In [ ]:
import os
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain

# Loading the OpenAI API Key

In [ ]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

# Loading The LLM (Language Model)


In [ ]:
llm = OpenAI(api_key=openai_api_key)

# Setting OpenAI Embeddings

In [ ]:
embed_model = OpenAIEmbeddings(api_key=openai_api_key)

# Loading Text Documents

In [ ]:
loader = TextLoader('monkey_story.txt') # to load text document
documents = loader.load()

# Splitting Text into Chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=128)
chunks = text_splitter.split_documents(documents)

# Creating a Vector Store (Chroma) from Documents

In [ ]:
vector_store = Chroma.from_documents(chunks, embed_model)

# Creating a Retriever


In [ ]:
retriever = vector_store.as_retriever()

# Creating a Retrieval Chain


In [ ]:
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

#  Retrieval-QA Chat Prompt


In [ ]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

# Combining Documents


In [ ]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

# Final Retrieval Chain


In [ ]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)    

# Invoking the Retrieval Chain


In [ ]:
retrieval_chain.invoke({"input": "name of monkeys."})